In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

In [10]:
train=pd.read_csv('Desktop/Advance deep learning/Data/padhai-mp-neuron-like-unlike-classification/train.csv')
test = pd.read_csv("Desktop/Advance deep learning/Data/padhai-mp-neuron-like-unlike-classification/test.csv")

In [11]:
test['Screen to Body Ratio (calculated)'].value_counts()

80.85 %    3
74.78 %    2
83.84 %    2
72.68 %    2
66.23 %    2
          ..
84.23 %    1
71.04 %    1
65.86 %    1
74.9 %     1
68.93 %    1
Name: Screen to Body Ratio (calculated), Length: 101, dtype: int64

In [12]:
columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display',
                         'Browser','Build Material','Co-Processor','Browser',
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features',
                         'Wireless Charging','USB OTG Support', 'Video Recording',
                         'Java','Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support',
                        'Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support', 'Custom UI',
                         'Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']

In [13]:
train.drop(set(columns_to_remove),axis=1,inplace=True)

In [14]:
test.drop(set(columns_to_remove),axis=1,inplace=True)

In [15]:
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 46
Number of data points in test: 119
Number of features in test: 45


In [16]:
train = train[(train.isnull().sum(axis=1) <= 12)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

Number of data points in train: 335
Number of features in train: 46


In [17]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def for_f(test):
    try:
        test = test.strip()
        test=test.replace('%','')
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

In [18]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_f)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [19]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 335
Number of features in train: 50
Number of data points in test: 119
Number of features in test: 49


In [20]:
columns_to_remove = ['User Available Storage','SIM Size','Processor','Autofocus','Aspect Ratio','Touch Screen'
                        ,'Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'Fingerprint Sensor','NFC','VoLTE','Chipset','Waterproof','Shooting Modes', 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type',
                        'Colours','Width','Model']
train.drop(set(columns_to_remove),axis=1,inplace=True)
test.drop(set(columns_to_remove),axis=1,inplace=True)
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 335
Number of features in train: 19
Number of data points in test: 119
Number of features in test: 18


In [21]:
train.drop('Expandable Memory',axis=1,inplace=True)
test.drop('Expandable Memory',axis=1,inplace=True)

In [22]:
test['Screen to Body Ratio (calculated)'].value_counts()

71.96    7
80.85    3
76.62    2
74.78    2
73.13    2
        ..
83.55    1
72.03    1
83.70    1
70.30    1
66.50    1
Name: Screen to Body Ratio (calculated), Length: 102, dtype: int64

In [23]:
train_ids = train['PhoneId']
test_ids = test['PhoneId']

In [24]:
cols=['Brand','SIM 2','SIM Slot(s)','os_name','Sim1','Num_cores','Screen to Body Ratio (calculated)','PhoneId']

In [25]:
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

In [26]:
combined = pd.concat([train[cols], test[cols]])

In [27]:
print(combined.shape)
print(combined.columns)

(454, 8)
Index(['PhoneId', 'Brand', 'SIM 2', 'SIM Slot(s)', 'os_name', 'Sim1',
       'Num_cores', 'Screen to Body Ratio (calculated)'],
      dtype='object')


In [28]:
combined = pd.get_dummies(combined)

In [29]:
print(combined.shape)
print(combined.columns)

(454, 76)
Index(['PhoneId', 'Screen to Body Ratio (calculated)', 'Brand_10.or',
       'Brand_Apple', 'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry',
       'Brand_Comio', 'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee',
       'Brand_Google', 'Brand_HTC', 'Brand_Honor', 'Brand_Huawei',
       'Brand_InFocus', 'Brand_Infinix', 'Brand_Intex', 'Brand_Itel',
       'Brand_Jivi', 'Brand_Karbonn', 'Brand_LG', 'Brand_Lava', 'Brand_LeEco',
       'Brand_Lenovo', 'Brand_Lyf', 'Brand_Meizu', 'Brand_Micromax',
       'Brand_Mobiistar', 'Brand_Moto', 'Brand_Motorola', 'Brand_Nokia',
       'Brand_Nubia', 'Brand_OPPO', 'Brand_OnePlus', 'Brand_Oppo',
       'Brand_Panasonic', 'Brand_Razer', 'Brand_Realme', 'Brand_Reliance',
       'Brand_Samsung', 'Brand_Sony', 'Brand_Spice', 'Brand_Tecno',
       'Brand_Ulefone', 'Brand_VOTO', 'Brand_Vivo', 'Brand_Xiaomi',
       'Brand_Xiaomi Poco', 'Brand_Yu', 'Brand_iVooMi', 'SIM 2_2G', 'SIM 2_3G',
       'SIM 2_4G', 'SIM 2_Other', 'SIM Slot(s)_Dual SIM, GSM+CDM

In [30]:
train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]
cols=['Capacity','Height','Internal Memory','Pixel Density','RAM','Resolution','Screen Size','Screen to Body Ratio (calculated)','Weight','Processor_frequency']

In [31]:
test_new.head()

,PhoneId,Screen to Body Ratio (calculated),Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,...,Sim1_3G,Sim1_4G,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa
0,3,80.85,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,11,81.60,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,13,83.84,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,16,83.84,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,19,77.43,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [32]:
cols=['Capacity','Height','Internal Memory','Pixel Density','RAM','Resolution','Screen Size','Screen to Body Ratio (calculated)','Weight','Processor_frequency']
for i in cols:
    train_new = train_new.merge(train[['PhoneId', i]], on='PhoneId')
    test_new = test_new.merge(test[['PhoneId', i]], on='PhoneId')

In [33]:
train_new.shape

(335, 86)

In [34]:
test_new.shape

(119, 86)

In [35]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [36]:
train_new.shape

(335, 87)

In [37]:
'''
cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)
print('1',combined.head())
combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)
print('2',combined.head())

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]
'''

"\ncols = list(test.columns)\ncols.remove('PhoneId')\ncols.insert(0, 'PhoneId')\n\ncombined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])\nprint(combined.shape)\nprint(combined.columns)\nprint('1',combined.head())\ncombined = pd.get_dummies(combined)\nprint(combined.shape)\nprint(combined.columns)\nprint('2',combined.head())\n\ntrain_new = combined[combined['PhoneId'].isin(train_ids)]\ntest_new = combined[combined['PhoneId'].isin(test_ids)]\n"

In [38]:
train_new.head()

,PhoneId,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,...,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency,Rating
0,0,80.68,0,0,0,0,0,0,0,0,...,157.9,64,403,4,20,6.26,80.68,182,1.8,4.5
1,1,80.85,0,0,0,0,0,0,0,0,...,156.2,32,271,3,8,6.20,80.85,168,1.8,4.5
2,2,83.68,0,0,0,0,0,0,0,0,...,157.0,32,409,3,25,6.30,83.68,168,2.1,4.4
3,4,74.78,0,0,0,0,0,0,0,0,...,159.8,64,411,4,24,6.00,74.78,169,2.2,4.3
4,5,84.23,0,0,0,0,0,0,0,0,...,160.4,64,396,4,16,6.50,84.23,175,2.2,4.4


In [39]:
test_new.head()

,PhoneId,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,3,80.85,0,0,0,0,0,0,0,0,...,4230,156.2,16,271,2,5,6.20,80.85,168,1.8
1,11,81.60,0,0,0,0,0,0,0,0,...,5000,156.0,64,402,4,12,6.20,81.60,205,1.8
2,13,83.84,0,0,0,0,0,0,0,0,...,3500,156.7,64,409,6,25,6.30,83.84,169,2.0
3,16,83.84,0,0,0,0,0,0,0,0,...,3500,156.7,64,409,4,16,6.30,83.84,169,2.0
4,19,77.43,0,0,0,0,0,0,0,0,...,4000,158.6,64,403,4,20,5.99,77.43,181,1.8


In [40]:
X=train_new.drop('Rating',axis=1)
y=train_new['Rating']

In [41]:
X_train = train_new.drop(['Rating','PhoneId'],axis = 1)
Y_train = train_new['Rating'].map(lambda x: 1 if x>=3.9 else 0)
X_test = test_new.drop(['PhoneId'],axis = 1)

In [42]:
'''
    for i in X_train.columns:
    maximum=max(X_train[i])
    minimum=min(X_train[i])
    for j in range(len(X_train[i])):
        X_train[i][j]=(X_train[i][j]-minimum)/(maximum-minimum)
'''

'\n    for i in X_train.columns:\n    maximum=max(X_train[i])\n    minimum=min(X_train[i])\n    for j in range(len(X_train[i])):\n        X_train[i][j]=(X_train[i][j]-minimum)/(maximum-minimum)\n'

In [43]:
X_train.isna()

,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
331,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
332,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
333,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
'''
for i in X_test.columns:
    maximum=max(X_test[i])
    minimum=min(X_test[i])
    for j in range(len(X_test[i])):
        X_test[i][j]=(X_test[i][j]-minimum)/(maximum-minimum)
        
'''

'\nfor i in X_test.columns:\n    maximum=max(X_test[i])\n    minimum=min(X_test[i])\n    for j in range(len(X_test[i])):\n        X_test[i][j]=(X_test[i][j]-minimum)/(maximum-minimum)\n        \n'

In [46]:
X_train.head()


,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,80.68,0,0,0,0,0,0,0,0,0,...,4000,157.9,64,403,4,20,6.26,80.68,182,1.8
1,80.85,0,0,0,0,0,0,0,0,0,...,4230,156.2,32,271,3,8,6.20,80.85,168,1.8
2,83.68,0,0,0,0,0,0,0,0,0,...,3500,157.0,32,409,3,25,6.30,83.68,168,2.1
3,74.78,0,0,0,0,0,0,0,0,0,...,3300,159.8,64,411,4,24,6.00,74.78,169,2.2
4,84.23,0,0,0,0,0,0,0,0,0,...,3750,160.4,64,396,4,16,6.50,84.23,175,2.2


In [47]:
X_train.drop('Brand_Billion',axis=1)

,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,80.68,0,0,0,0,0,0,0,0,0,...,4000,157.9,64,403,4,20,6.26,80.68,182,1.8
1,80.85,0,0,0,0,0,0,0,0,0,...,4230,156.2,32,271,3,8,6.20,80.85,168,1.8
2,83.68,0,0,0,0,0,0,0,0,0,...,3500,157.0,32,409,3,25,6.30,83.68,168,2.1
3,74.78,0,0,0,0,0,0,0,0,0,...,3300,159.8,64,411,4,24,6.00,74.78,169,2.2
4,84.23,0,0,0,0,0,0,0,0,0,...,3750,160.4,64,396,4,16,6.50,84.23,175,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,67.20,0,0,0,0,0,0,0,0,0,...,2420,142.7,16,220,1,5,5.00,67.20,141,1.2
331,69.95,0,0,0,0,0,0,0,0,0,...,3000,156.5,64,401,4,13,5.50,69.95,165,1.8
332,68.52,0,0,0,0,0,0,0,0,0,...,3000,149.7,16,282,3,8,5.20,68.52,160,1.4
333,63.93,0,0,0,0,0,0,0,0,0,...,1800,132.0,8,218,1,2,4.50,63.93,156,1.3


In [48]:
X_test.dropna(inplace=True)

In [49]:
X_test

,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,80.85,0,0,0,0,0,0,0,0,0,...,4230,156.2,16,271,2,5,6.20,80.85,168,1.8
1,81.60,0,0,0,0,0,0,0,0,0,...,5000,156.0,64,402,4,12,6.20,81.60,205,1.8
2,83.84,0,0,0,0,0,0,0,0,0,...,3500,156.7,64,409,6,25,6.30,83.84,169,2.0
3,83.84,0,0,0,0,0,0,0,0,0,...,3500,156.7,64,409,4,16,6.30,83.84,169,2.0
4,77.43,0,0,0,0,0,0,0,0,0,...,4000,158.6,64,403,4,20,5.99,77.43,181,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,73.18,0,0,0,0,0,0,1,0,0,...,4000,152.0,32,401,4,8,5.50,73.18,167,1.8
115,83.99,0,0,0,0,0,0,0,0,0,...,3500,158.1,256,531,6,8,6.20,83.99,189,2.7
116,70.72,0,0,0,0,0,0,0,0,0,...,3000,151.8,8,267,3,5,5.50,70.72,171,1.2
117,68.66,0,0,0,0,0,1,0,0,0,...,2700,149.2,32,282,2,8,5.20,68.66,159,1.3


In [50]:
class Perceptron:
  def __init__(self):
    self.b = None
    self.w = None
  def model(self,x):
    return 1 if (np.dot(self.w,x) >= self.b) else 0
  
  def predict(self,X):
    Y = []
    for x in X:
      pred = self.model(x)
      Y.append(pred)
    return np.array(Y)
  
  def fit(self, X , Y ,epochs = 1,lr = 0.015):
    accuracy = {}
    max_accuracy = 0
    wt_matrix = []
    self.w = np.ones(X.shape[1])
    self.b = 0
    for i in range(epochs):
      for x,y in zip(X,Y):
        Y_pred = self.model(x)
        m = X.shape[1]
        if y==1 and Y_pred == 0:
          self.w = self.w + lr*x/m
          self.b+=1
        if y==0 and Y_pred == 1:
          self.w = self.w - lr*x/m
          self.b-=1   
      wt_matrix.append(self.w)
      
      accuracy[i] = accuracy_score(self.predict(X),Y)
      if (accuracy[i] > max_accuracy):
        max_accuracy = accuracy[i]
        chkpntb = self.b
        chkpntw = self.w
    
    self.w = chkpntw
    self.b = chkpntb
    
    print(max_accuracy)
    return np.array(wt_matrix)

In [53]:
perceptron = Perceptron()


In [55]:
perceptron.fit(X_train.values,Y_train.values,epochs = 1000,lr = 0.01)

0.8


array([[ 0.94867412,  0.99976471,  1.        , ...,  0.94867412,
         0.88211765,  0.99886   ],
       [ 0.90675059,  0.99952941,  1.00011765, ...,  0.90675059,
         0.78705882,  0.99828941],
       [ 0.86790235,  0.99929412,  1.00011765, ...,  0.86790235,
         0.70411765,  0.99788706],
       ...,
       [ 3.30883059,  0.76517647,  1.03411765, ...,  3.30883059,
        -6.51105882,  1.56803647],
       [ 3.30631294,  0.76494118,  1.03411765, ...,  3.30631294,
        -6.50647059,  1.56845176],
       [ 3.31194588,  0.76470588,  1.03411765, ...,  3.31194588,
        -6.48411765,  1.56899647]])

In [56]:
for i in X_train.columns:
    maximum=max(X_train[i])
    minimum=min(X_train[i])
    if(maximum!=minimum):
        for j in range(len(X_train[i])):
            X_train[i][j]=(X_train[i][j]-minimum)/(maximum-minimum)

In [57]:
X_train.isna()

,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
331,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
332,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
333,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [58]:
for i in X_test.columns:
    maximum=max(X_test[i])
    minimum=min(X_test[i])
    if maximum!=minimum:
        for j in range(len(X_test[i])):
            X_test[i][j]=(X_test[i][j]-minimum)/(maximum-minimum)

In [59]:
X_test.isna()

,Screen to Body Ratio (calculated)_x,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,...,Capacity,Height,Internal Memory,Pixel Density,RAM,Resolution,Screen Size,Screen to Body Ratio (calculated)_y,Weight,Processor_frequency
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
116,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
117,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [60]:
class Perceptron:
  def __init__(self):
    self.b = None
    self.w = None
  def model(self,x):
    return 1 if (np.dot(self.w,x) >= self.b) else 0
  
  def predict(self,X):
    Y = []
    for x in X:
      pred = self.model(x)
      Y.append(pred)
    return np.array(Y)
  
  def fit(self, X , Y ,epochs = 1,lr = 0.015):
    accuracy = {}
    max_accuracy = 0
    wt_matrix = []
    self.w = np.ones(X.shape[1])
    self.b = 0
    for i in range(epochs):
      for x,y in zip(X,Y):
        Y_pred = self.model(x)
        m = X.shape[1]
        if y==1 and Y_pred == 0:
          self.w = self.w + lr*x/m
          self.b+=1
        if y==0 and Y_pred == 1:
          self.w = self.w - lr*x/m
          self.b-=1   
      wt_matrix.append(self.w)
      
      accuracy[i] = accuracy_score(self.predict(X),Y)
      if (accuracy[i] > max_accuracy):
        max_accuracy = accuracy[i]
        chkpntb = self.b
        chkpntw = self.w
    
    self.w = chkpntw
    self.b = chkpntb
    
    print(max_accuracy)
    return np.array(wt_matrix)

In [61]:
perceptron = Perceptron()

In [62]:
perceptron.fit(X_train.values,Y_train.values,epochs = 1000,lr = 0.01)

0.7850746268656716


array([[ 0.99401712,  0.99976471,  1.        , ...,  0.99401712,
         1.        ,  0.99740458],
       [ 0.98803423,  0.99952941,  1.        , ...,  0.98803423,
         1.        ,  0.99480915],
       [ 0.98205135,  0.99929412,  1.        , ...,  0.98205135,
         1.        ,  0.99221373],
       ...,
       [-4.97091793,  0.76517647,  1.        , ..., -4.97091793,
         1.        , -1.59023399],
       [-4.97690081,  0.76494118,  1.        , ..., -4.97690081,
         1.        , -1.59282941],
       [-4.98288369,  0.76470588,  1.        , ..., -4.98288369,
         1.        , -1.59542484]])